###**Installing required Libraries**

In [ ]:
!pip install transformers datasets accelerate seqeval==0.0.12

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.5/7.5 MB 17.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.3/519.3 kB 24.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 251.2/251.2 kB 16.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 29.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 49.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 52.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 13.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 25.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 18.2 MB/s eta 0:00:00
  Created wheel for seqeval: filename=seqeval-0.0.12-py3-none-any.whl size=7414 sha256=20fe6ed6f8e7d073b6729ae2a00e960c48987f9cd7b9dffa0bc5aa05e3eb3dda
  Stored in directory: /root/.cache/pip/wheels/6c/6c/fc/7076d687ba54f32c7be7eaaded97

###**Importing required Modules**

In [ ]:
from transformers import AutoTokenizer
import pandas as pd
import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader, RandomSampler, SequentialSampler, default_collate
from torch.optim import Adam
from datasets import Dataset
import json
from sklearn.model_selection import train_test_split
from tqdm import trange
from seqeval.metrics import classification_report
from sklearn.metrics import confusion_matrix

###**Mounting Drive and loading the data**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
df = pd.read_json('/content/drive/MyDrive/Resumes_tagged.json') #reading data from json to pandas dataframe

##**Exploring the data**

In [ ]:
df.shape

(311, 3)

In [ ]:
df.head()

,tags,id,text
0,"[{'end': 49, 'start': 28, 'tag': 'Email'}, {'e...",0,Name: Yaswanth Gade\nEmp lD: yaswant.gade@gmai...
1,"[{'end': 7, 'start': 0, 'tag': 'FullName'}, {'...",1,YASHASRI\nM\nmyashu794@gmail.com\n+91-85230790...
2,"[{'end': 16, 'start': 0, 'tag': 'FullName'}, {...",2,"Vishwadeep Tiwari\nContact no: +91 9455926562,..."
3,"[{'end': 13, 'start': 0, 'tag': 'FullName'}, {...",3,VISHNU KUMAR V\nSoftware Engineer with 4+ year...
4,"[{'end': 7, 'start': 0, 'tag': 'FullName'}, {'...",4,Vishnu C\nvishnucmayyil@gmail.com\n+91-9633 34...


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 311 entries, 0 to 310
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   tags    311 non-null    object
 1   id      311 non-null    int64 
 2   text    311 non-null    object
dtypes: int64(1), object(2)
memory usage: 7.4+ KB


Through an initial observation. it is noted that the has 3 components which are

1.   Text for each resume (object)
2.   ID of resume (int)
3.   Tags (object)

In [ ]:
df.loc[0]['tags'][0].keys()

dict_keys(['end', 'start', 'tag'])

Each tag has 3 elements which are
1.   Start - Starting index of tagged text
2.   End - Ending index of the tagged text
3.   tag - Name of the tag

In [ ]:
df.describe()

,id
count,311.000000
mean,155.000000
std,89.922189
min,0.000000
25%,77.500000
50%,155.000000
75%,232.500000
max,310.000000


In [ ]:
df.columns

Index(['tags', 'id', 'text'], dtype='object')

In [ ]:
df.loc[2]

tags    [{'end': 16, 'start': 0, 'tag': 'FullName'}, {...
id                                                      2
text    Vishwadeep Tiwari\nContact no: +91 9455926562,...
Name: 2, dtype: object

In [ ]:
#Checking the labelled data

temp = df.loc[1]
for tag_item in temp["tags"]:
    print(tag_item["tag"], "-", temp["text"][tag_item["start"]: tag_item["end"]+1])
    print("\n")

FullName - YASHASRI


FirstName - YASHASRI


FullName - Yashasri Munuganti


FirstName - Yashasri


LastName - Munuganti


FullName - Yashasri M


FirstName - Yashasri


LastName - M


EducationDegree - B.Tech


EducationSpecialization - ECE


EducationUniversity - MINA Institute of Engg&Tech (JNTU)


EducationMarks - 68.96%


EducationYear - 2013


EducationText - B.Tech (ECE) from MINA Institute of Engg&Tech (JNTU), with 68.96% in 2013.


EducationDegree - Intermediate


EducationUniversity - Kakathiya Junior College


EducationLocation - Nakrekal


EducationMarks - 68.9%


EducationYear - 2009


EducationText - Intermediate from Kakathiya Junior College, Nakrekal with 68.9% in 2009.


EducationDegree - SSC


EducationUniversity - Nethaji High School


EducationLocation - Nakrekal


EducationMarks - 75.04%


EducationYear - 2007


EducationText - SSC from Nethaji High School, Nakrekal with 75.04% in 2007.


Email - myashu794@gmail.com


Mobile - +91-8523079060


Summary - Having 4 Ye

In [ ]:
# Splitting data into train and test data

train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)

In [ ]:
#Converting the pandas datafranes into pytorch dataset objects for further preprocessing

training_dataset = Dataset.from_pandas(train_df)
test_dataset = Dataset.from_pandas(test_df)

In [ ]:
training_dataset.features

{'tags': [{'end': Value(dtype='int64', id=None),
   'start': Value(dtype='int64', id=None),
   'tag': Value(dtype='string', id=None)}],
 'id': Value(dtype='int64', id=None),
 'text': Value(dtype='string', id=None),
 '__index_level_0__': Value(dtype='int64', id=None)}

In [ ]:
training_dataset.shape

(248, 4)

In [ ]:
test_dataset.features

{'tags': [{'end': Value(dtype='int64', id=None),
   'start': Value(dtype='int64', id=None),
   'tag': Value(dtype='string', id=None)}],
 'id': Value(dtype='int64', id=None),
 'text': Value(dtype='string', id=None),
 '__index_level_0__': Value(dtype='int64', id=None)}

In [ ]:
test_dataset.shape

(63, 4)

##**Creating required entity lists to specify the various types of tags to be used**

In [ ]:
listofentities = {
    'Achievements','Address','CertificateDate','CertificateLocation','CertificateORG','CertificateSpecialization','CertificationText','Certifications','CertificationtTitle',
    'CurrentCTC','CurrentLocation','Date','DateOfBirth','Declaration','Education','EducationDegree','EducationInstitute','EducationLocation','EducationMarks','EducationSpecialization',
    'EducationText','EducationUniversity','EducationYear','Educations','Email','Experience','ExtraCurricularActivities','FatherName','FirstName','FullName','Gender','Hobbies',
    'JobCategory','JobClientORG','JobCompany','JobDescription','JobLocation','JobPeriod','JobProfileRoleDesignation','JobText','LanguageKnown','LastName','MaritalStatus','MiddleName',
    'Mobile','MotherName','Name','Nationality','NotitePeriod','O','Objective','Objectives','PreferredLocation','Project','ProjectClient','ProjectDuration','ProjectName','ProjectSkills',
    'ProjectTeamSize','ProjectText','ProjectDescription','References','SkillBlock','SkillKeywords','Summary','WebSite'
 }

In [ ]:
entitylist = sorted(list(listofentities))
entitydict = {}
for i in range(len(entitylist)):
  entitydict[entitylist[i]] = i
entitydict

{'Achievements': 0,
 'Address': 1,
 'CertificateDate': 2,
 'CertificateLocation': 3,
 'CertificateORG': 4,
 'CertificateSpecialization': 5,
 'CertificationText': 6,
 'Certifications': 7,
 'CertificationtTitle': 8,
 'CurrentCTC': 9,
 'CurrentLocation': 10,
 'Date': 11,
 'DateOfBirth': 12,
 'Declaration': 13,
 'Education': 14,
 'EducationDegree': 15,
 'EducationInstitute': 16,
 'EducationLocation': 17,
 'EducationMarks': 18,
 'EducationSpecialization': 19,
 'EducationText': 20,
 'EducationUniversity': 21,
 'EducationYear': 22,
 'Educations': 23,
 'Email': 24,
 'Experience': 25,
 'ExtraCurricularActivities': 26,
 'FatherName': 27,
 'FirstName': 28,
 'FullName': 29,
 'Gender': 30,
 'Hobbies': 31,
 'JobCategory': 32,
 'JobClientORG': 33,
 'JobCompany': 34,
 'JobDescription': 35,
 'JobLocation': 36,
 'JobPeriod': 37,
 'JobProfileRoleDesignation': 38,
 'JobText': 39,
 'LanguageKnown': 40,
 'LastName': 41,
 'MaritalStatus': 42,
 'MiddleName': 43,
 'Mobile': 44,
 'MotherName': 45,
 'Name': 46,


In [ ]:
#Creating the tag2id and id2tag dictionaries which are used to encode and decode labels

tag2id = entitydict
id2tag = {v:k for k, v in tag2id.items()}
id2tag

{0: 'Achievements',
 1: 'Address',
 2: 'CertificateDate',
 3: 'CertificateLocation',
 4: 'CertificateORG',
 5: 'CertificateSpecialization',
 6: 'CertificationText',
 7: 'Certifications',
 8: 'CertificationtTitle',
 9: 'CurrentCTC',
 10: 'CurrentLocation',
 11: 'Date',
 12: 'DateOfBirth',
 13: 'Declaration',
 14: 'Education',
 15: 'EducationDegree',
 16: 'EducationInstitute',
 17: 'EducationLocation',
 18: 'EducationMarks',
 19: 'EducationSpecialization',
 20: 'EducationText',
 21: 'EducationUniversity',
 22: 'EducationYear',
 23: 'Educations',
 24: 'Email',
 25: 'Experience',
 26: 'ExtraCurricularActivities',
 27: 'FatherName',
 28: 'FirstName',
 29: 'FullName',
 30: 'Gender',
 31: 'Hobbies',
 32: 'JobCategory',
 33: 'JobClientORG',
 34: 'JobCompany',
 35: 'JobDescription',
 36: 'JobLocation',
 37: 'JobPeriod',
 38: 'JobProfileRoleDesignation',
 39: 'JobText',
 40: 'LanguageKnown',
 41: 'LastName',
 42: 'MaritalStatus',
 43: 'MiddleName',
 44: 'Mobile',
 45: 'MotherName',
 46: 'Name',


##**Building the model utlities**

## **create_datalist(dataset)**

This function creates a text,entity pair list of data from a given dataset

**INPUT** - takes a dataset as input, where the dataset is a list of dictionaries with each dictionary having 2 elements- the resume text and a list of dictionaries containing the `tag`, `start` and `end`.

**OUTPUT** - The function processes each entry in the dataset and creates a new list `datalist`, which is a list of lists with each list containing 2 elements the resume_text and a dictionary having a single element `entities` which contains a list of lists each of which has 3 elements `tag`, `start` and `end`.


###**Function**###

1.   The function initializes an empty list called `datalist` to store the processed data entries.

2.   It iterates through each data item in the input dataset and extracts the text field ('text') and removes any newline characters ("\n") by replacing them with spaces.

3.  It retrieves the list of tags ('tags') from the data item. Each tag is expected to be a dictionary with keys `start`, `end`, and `tag`.

4.  For each tag in the list, it extracts the `start`, `end`, and `tag` information.

5.  It checks if the label has leading or trailing spaces by comparing the length of the label with the length of the label after stripping leading and trailing spaces and adjusts the start and end positions accordingly to remove the extra spaces.

6.  It appends the entity information start, end and label as a list `[start, end + 1, label]` to the entities list.

7.  It appends the processed data entry (text and entities dictionary) as a list `[text, {"entities": entities}]` to the datalist.


In [ ]:
def create_datalist(dataset):
    datalist = []
    for data in dataset:
        text = data['text'].replace("\n"," ")
        entities = []
        tags = data['tags']
        for tag in tags:
            start = tag['start']
            end = tag['end']
            label = tag['tag']
            lstrip_diff = len(label) - len(label.lstrip())
            rstrip_diff = len(label) - len(label.rstrip())
            if lstrip_diff != 0:
                start = start + lstrip_diff
            if rstrip_diff != 0:
                end = end - rstrip_diff
            entities.append([start, end + 1, label])
        datalist.append([text, {"entities": entities}])
    return datalist

train = create_datalist(training_dataset)
test = create_datalist(test_dataset)
print(train[0][0])
print(train[0][1])

CURRICULUM VITAE Srinivas Naik G Mobile: +919008561834 E-mail: srini95143@gmail.com CAREER OBJECTIVE: To be an integral part of a dynamic and fast-paced organization, to work for the organization by continuous learning and utilizing the skills necessary for achieving the organizational goals. EDUCATION: 2017 B.E. (Environmental Engineering) from JSS SJCE Mysuru with CGPA 7.73. 2013 12th from SMPU Dharwad, with 56%. 2011 10th from BGSREPHS Bananduru, Ramanagar with 76.48%. NEBOSH IGC International certificate in Safety. Registered for Advance Diploma in Industrial Safety (ADIS accredited to MSBTE) PROFESSIONAL EXPERIENCE: LM WINDPOWER a GE Renewable Energy Business: (Present Company) Key Role – Advanced Engineer – EHS: 1. Safety induction training for new joiners & Visitors. 2. Internal safety Genba Walk - Planning and coordinating with CFT throughout plant. 3. Conducting classroom EHS internal training for employees on EHS. 4. Promote and providing safety awareness programs to employee

In [ ]:
resticted_lables = ["UNKNOWN", "O"]

##**get_label(offset, labels)**##

This function returns the label of the token based on whether it is part of any entity span or not.

The function takes two parameters as **INPUT**:

**offset**: A tuple representing the character offset of the current token (start position, end position).

**labels**: A list of entities, where each entity is represented as a tuple containing its character span (start position, end position) and its label (a string).

###**FUNCTION**###

1.  The function first checks if the offset is (0, 0). If it is, it means that the current token is a special token eg- `[CLS]`, `[PAD]`, etc. and not part of any entity. In this case, the function returns the label 'O', indicating that the token is not part of any entity.If the offset is not (0, 0), it means the token is a regular word/token in the input text, and the function needs to find its label.

2.  The function iterates through the list of labels, where each label is a tuple representing an entity's character span and label.

3.  For each label, it checks if the offset of the current token lies within the span of the entity and if the offset is inside an entity span, it means the token is part of that entity, and the function returns the corresponding label for that entity.

4.  If the offset does not match any entity span, the function returns the label 'O', indicating that the token is not part of any entity.

In [ ]:
def get_label(offset, labels):
    if offset[0] == 0 and offset[1] == 0:
        return 'O'
    for label in labels:
        if offset[1] >= label[0] and offset[0] <= label[1]:
            return label[2]
    return 'O'

##**process(data, tokenizer, tag2id, max_len)**##

this function takes input text data, tokenizes it using a given tokenizer, maps the original labels to label ids, adds padding and truncation, and returns the processed data.

The function takes **4** parameters as **INPUT**:

* **data**: resume data in the form of a string which is to be tokenized and processed.

* **tokenizer**: This is the given tokenizer which is to be used to tokenize the textual data.

* **tag2id**: A dictionary containing the `labels` and their corresponding `label_ids`.

* **max_len**: maximum length of tokenized data.

###**FUNCTION**###

1.  The `tokenizer.encode_plus` function is used to tokenize the given text data and return tokens as well as other information like the attention mask, token type IDs, and offset mapping. this function accepts text (`data[0]`) from the input element alongwith the `max_len` and `return_offsets_mapping=True` and returns a dictionary with elements  -
*    `input_ids` (ids of tokens as returned by tokenizer)
*    `token_type_ids` (which sequence tokens belong to, all 0 in this case)
*    `attention_mask` (describes which tokens the model should consider)

2.  The function then extracts the actual labels of tokens `orig_labels` and the label ids for these tokens `labels`

3.  The function ten adds padding to each of the output parameters to create equal length arrays which are to be further processed before being used by the model

4. Lastly, the function returns a dictionary with the elements `orig_labels`, `labels`, `input_ids`, `token_type_ids`, `attention_mask`

In [ ]:
def process(data, tokenizer, tag2id, max_len):
  tokenized = tokenizer.encode_plus(
      data[0],
      max_length= max_len,
      return_offsets_mapping=True) #tokenizing the given text from the resume

  padding_length = max_len - len(tokenized['input_ids'])

  label_data = {'orig_labels': [], 'labels': []}
  labels = data[1]['entities']
  labels.reverse()

  #Extracting the labels and label_ids for the given tokens
  for offset in tokenized['offset_mapping']:
      label = get_label(offset, labels)
      label_data['orig_labels'].append(label)
      label_data['labels'].append(tag2id[label])

  #Applying padding to create same sized arrays
  label_data['labels'] = label_data['labels'] + ([0] * padding_length)
  label_data['input_ids'] = tokenized['input_ids'] + ([0] * padding_length)
  label_data['token_type_ids'] = tokenized['token_type_ids'] + ([0] * padding_length)
  label_data['attention_mask'] = tokenized['attention_mask'] + ([0] * padding_length)

  label_data

  return label_data

##**ResumeDataset(Dataset)**##

This is used to define a custom dataset which is used to create a dataloader for implementing a NER task

1. `__init__(self, resume, tokenizer, tag2id, max_len)`

>The ResumeDataset object is initialized with the 4 parameters:
* **resume**: A list of dictionaries, where each dictionary item contains the resume text and named entity annotations.
* **tokenizer**: The tokenizer object used to tokenize the text.
* **tag2id**: A dictionary that maps labels to their corresponding label_ids.
* **max_len**: The maximum length of the tokenized resume text after padding.

2. `__len__(self)`

>The `len` method is used to calculate the number of resume elements present in the dataset

3. `__getitem__(self, idx)`

>This method is used to access individual elements of a dataset from a given list of index values.
1. The method takes `idx` as input, which can be a single integer index or a list of integer indices.
2. The method then retrieves the resue elements based on the idx values and processes the resume data using the `process()` function and returns the output as a list.

4. `__getitems__(self, keys)`

> This method is called to get a batch of data samples from the dataset.
1. The method takes `keys` as input, which is a list of integer indices and then calls the `__getitem__()` method to process the resumes according to the given indices.
2. The method returns a list of processed batches, each containing tokenized resume data and other details.


In [ ]:
class ResumeDataset(Dataset):
    def __init__(self, resume, tokenizer, tag2id, max_len):
        self.resume = resume
        self.tokenizer = tokenizer
        self.tag2id = tag2id
        self.max_len = max_len

    def __len__(self):
        return len(self.resume)

    def __getitem__(self, idx):
      result = []
      for i in idx:
        data = self.resume[i]
        result.append(process(data, self.tokenizer, self.tag2id, self.max_len))
      return result

    def __getitems__(self, keys):
      batch = self.__getitem__(keys)
      return batch


##**get_hyperparameters(model, ff)**##

This function generates parameter groups for the optimizer during the model training

The function takes **2** arguments as **INPUT**:

* **model:** The model for which the optimizer's parameter groups are generated.

* **ff:** A boolean flag indicating whether full finetuning is enabled or not.


###**FUNCTION**###

1. Firstly, the function checks if full-finetuning(ff) flag is true:

>* If ff is True , it prepares the optimizer's parameter groups for fine-tuning the entire model. the model parameters are split into 2 groups:
>>1. **no_decay**: This group contains parameters that do not undergo weight decay. Typically, these parameters include bias, gamma and beta.
>>2. **decay**: This group contains parameters that will undergo weight decay during optimization.
* If ff is False, function assumes that only the classifier head (final layer(s)) of the model will be fine-tuned, and the rest of the model will be frozen. Thus, function only includes the classifier's parameters in the optimizer's parameter groups.

2.  Lastly, the function returns the list of optimizer parameter groups, which will be used during training.


In [ ]:
def get_hyperparameters(model, ff):

    # ff: full_finetuning
    if ff:
        param_optimizer = list(model.named_parameters())
        no_decay = ["bias", "gamma", "beta"]
        optimizer_grouped_parameters = [
            {
                "params": [
                    p for n, p in param_optimizer if not any(nd in n for nd in no_decay) #adding weight decay parameters with decay value = 0.01
                ],
                "weight_decay_rate": 0.01,
            },
            {
                "params": [
                    p for n, p in param_optimizer if any(nd in n for nd in no_decay) #adding non weight decay parameters with decay value = 0
                ],
                "weight_decay_rate": 0.0,
            },
        ]
    else:
        param_optimizer = list(model.classifier.named_parameters())
        optimizer_grouped_parameters = [
            {"params": [p for n, p in param_optimizer]}]

    return optimizer_grouped_parameters

##**get_special_tokens(tokenizer, tag2id)**##

This function is used to obtain the token ids for the special tokens `[PAD]`, `[SEP]`, `[CLS]`, and `O`

The function takes **2** arguments as **INPUT**:

* **tokenizer**: The tokenizer used to tokenize the text.

* **tag2id**: A dictionary that maps labels to their corresponding label_ids.


###**FUNCTION**###

1. The function uses the `get_vocab(tokenizer)` method to extract the vocabulary of the tokenizer in the form of a dictionary with tokens and their mapped token_ids

2.  The function then extracts the token ids of the  `[PAD]`, `[SEP]`, `[CLS]`, and `O` tokens.

3. Lastly, the function returns the respective token ids


In [ ]:
def get_special_tokens(tokenizer, tag2id):
    vocab = tokenizer.get_vocab()
    pad = vocab["[PAD]"]
    sep = vocab["[SEP]"]
    cls = vocab["[CLS]"]
    o = tag2id["O"]

    return pad, sep, cls, o

In [ ]:
#Calculates the accuracy of the predictions
def flat_accuracy(valid_tags, pred_tags):
    return (np.array(valid_tags) == np.array(pred_tags)).mean()

In [ ]:
def train_eval_model(
    model,
    tokenizer,
    optimizer,
    epochs,
    id2tag,
    tag2id,
    max_grad_norm,
    device,
    train_dataloader,
    valid_dataloader
):

    pad, sep, cls, o = get_special_tokens(tokenizer, tag2id)

    epoch = 0
    for t in trange(epochs, desc="Epoch"):
        epoch += 1

        # Training loop
        model.train()
        tr_loss, tr_accuracy = 0, 0
        nb_tr_examples, nb_tr_steps = 0, 0
        tr_preds, tr_labels = [], []

        for step, batch in enumerate(train_dataloader):
            # Add batch to gpu
            b_input_ids = torch.tensor([item['input_ids'] for item in batch])
            b_input_mask = torch.tensor([item['attention_mask'] for item in batch])
            b_labels = torch.tensor([item['labels'] for item in batch])

            #batch = tuple(b.to(device) for b in batch)
            b_input_ids, b_input_mask, b_labels = b_input_ids.to(device), b_input_mask.to(device), b_labels.to(device)

            # Forward pass
            outputs = model(
                b_input_ids,
                token_type_ids=None,
                attention_mask=b_input_mask,
                labels=b_labels
            ) #Passing tensors as inputs to model and obtaining result
            loss, tr_logits = outputs[:2] #extracting loss and logits from model output

            # Backward pass
            loss.backward()

            # Compute train loss
            tr_loss += loss.item()
            nb_tr_examples += b_input_ids.size(0) #tracking number of processed training examples
            nb_tr_steps += 1 #tracking number of steps(epochs) completed

            # Subset out unwanted predictions on CLS/PAD/SEP tokens
            preds_mask = (
                (b_input_ids != cls)
                & (b_input_ids != pad)
                & (b_input_ids != sep)
            )

            #Calculating predictions
            tr_logits = tr_logits.cpu().detach().numpy() #used to convert toch tensor to numpy array after removing computational graph
            tr_label_ids = torch.masked_select(b_labels, (preds_mask == 1)) #used to select the label_id with threshold value 1
            preds_mask = preds_mask.cpu().detach().numpy() #mask which is used to filter out the special tokens
            tr_batch_preds = np.argmax(tr_logits[preds_mask.squeeze()], axis=1) #applies argmax on the logits of relevant tokens to find the value of the most probable label for each token
            tr_batch_labels = tr_label_ids.to("cpu").numpy() #array of actual label_ids
            tr_preds.extend(tr_batch_preds)
            tr_labels.extend(tr_batch_labels)

            # Compute training accuracy
            tmp_tr_accuracy = flat_accuracy(tr_batch_labels, tr_batch_preds) # calculates the accuracy
            tr_accuracy += tmp_tr_accuracy

            # Gradient clipping
            torch.nn.utils.clip_grad_norm_(
                parameters=model.parameters(), max_norm=max_grad_norm
            ) #performs gradient clipping to restrict gradients to a threshold value so as to prevent exploding gradients

            # Update parameters
            optimizer.step()
            model.zero_grad()

        #Calculating loss and accuracy for epoch
        tr_loss = tr_loss / nb_tr_steps
        tr_accuracy = tr_accuracy / nb_tr_steps

        # Print training loss and accuracy per epoch
        print(f"\nTrain loss: {tr_loss}")
        print(f"Train accuracy: {tr_accuracy}")


      #---------------------------------------------------------#


        # Evaluation loop
        model.eval()
        eval_loss, eval_accuracy = 0, 0
        nb_eval_steps, nb_eval_examples = 0, 0
        predictions, true_labels = [], []

        for batch in valid_dataloader:

            b_input_ids = torch.tensor([item['input_ids'] for item in batch])
            b_input_mask = torch.tensor([item['attention_mask'] for item in batch])
            b_labels = torch.tensor([item['labels'] for item in batch])

            b_input_ids, b_input_mask, b_labels = b_input_ids.to(device), b_input_mask.to(device), b_labels.to(device)

            with torch.no_grad():
                outputs = model(
                    b_input_ids,
                    token_type_ids=None,
                    attention_mask=b_input_mask,
                    labels=b_labels,
                ) #Passing tensors as inputs to model and obtaining result
                tmp_eval_loss, logits = outputs[:2] #extracting loss and logits from model output


            nb_eval_examples += b_input_ids.size(0) #tracking number of processed training examples
            nb_eval_steps += 1 #tracking number of steps(epochs) completed

            # Subset out unwanted predictions on CLS/PAD/SEP tokens
            preds_mask = (
                (b_input_ids != cls)
                & (b_input_ids != pad)
                & (b_input_ids != sep)
            )

            #Calculating predictions
            logits = logits.cpu().detach().numpy() #used to convert toch tensor to numpy array after removing computational graph
            label_ids = torch.masked_select(b_labels, (preds_mask == 1)) #used to select the label_id with threshold value 1
            preds_mask = preds_mask.cpu().detach().numpy() #used to filter out the special tokens
            val_batch_preds = np.argmax(logits[preds_mask.squeeze()], axis=1) #applies argmax on the logits of relevant tokens to find the value of the most probable label for the token
            val_batch_labels = label_ids.to("cpu").numpy() #array of actual label_ids
            predictions.extend(val_batch_preds)
            true_labels.extend(val_batch_labels)

            tmp_eval_accuracy = flat_accuracy(val_batch_labels, val_batch_preds) # calculates the accuracy


            #Calculating loss and accuracy for epoch
            eval_loss += tmp_eval_loss.mean().item()
            eval_accuracy += tmp_eval_accuracy


        # Evaluate loss, acc, and class. report
        eval_loss = eval_loss / nb_eval_steps
        eval_accuracy = eval_accuracy / nb_eval_steps
        pred_tags = [id2tag[i] for i in predictions] #extracts predicted tags
        true_tags = [id2tag[i] for i in true_labels] #extracts actual tags
        cl_report = classification_report(true_tags, pred_tags)


        # Report metrics
        print(f"\nValidation loss: {eval_loss}")
        print(f"Validation Accuracy: {eval_accuracy}")
        print(f"Classification Report:\n {cl_report}")

##**Loading Model and Tokenizer**

In [ ]:
from transformers import AutoTokenizer, BertForTokenClassification, AutoModelForTokenClassification, BertTokenizerFast

longformer_model = "kiddothe2b/longformer-mini-1024"
resume_bert = "bert-base-uncased"

tokenizer = AutoTokenizer.from_pretrained(resume_bert) #loading tokenizer
tokenizer = BertTokenizerFast('/content/drive/MyDrive/vocab.txt', lowercase=True)
model = BertForTokenClassification.from_pretrained(resume_bert, num_labels=len(tag2id)) #loading model

tokenizer = AutoTokenizer.from_pretrained(longformer_model)
model = AutoModelForTokenClassification.from_pretrained(longformer_model, num_labels=len(tag2id))

Some weights of BertForTokenClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Some weights of LongformerForTokenClassification were not initialized from the model checkpoint at kiddothe2b/longformer-mini-1024 and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


##**Specifying the training arguments**

In [ ]:
#Training Arguments
MAX_LEN = 1000
EPOCHS = 20
MAX_GRAD_NORM = 1.0
TOKENIZER = tokenizer
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

##**Preparing training and evaluation Data**

In [ ]:
#Creating train and validation datasets
train_ds = ResumeDataset(train, TOKENIZER, tag2id, MAX_LEN)
val_ds = ResumeDataset(test, TOKENIZER, tag2id, MAX_LEN)

In [ ]:
len(train_ds.resume)

248

In [ ]:
len(val_ds.resume)

63

In [ ]:
#Creating dataloaders for train and validation datasets
train_dl = DataLoader(train_ds, batch_size=8, collate_fn = lambda x: x)
val_dl = DataLoader(val_ds, batch_size=6, collate_fn = lambda x: x )

In [ ]:
len(train_dl)

31

In [ ]:
#Exploring Dataloader elements
batch = next(iter(train_dl))
keys = batch[0].keys()
print(keys)
print((batch[0]['orig_labels']))
print(batch[0]['labels'])
print(batch[0]['input_ids'])
print(batch[0]['token_type_ids'])
print(batch[0]['attention_mask'])

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


dict_keys(['orig_labels', 'labels', 'input_ids', 'token_type_ids', 'attention_mask'])
['O', 'O', 'O', 'O', 'FirstName', 'FirstName', 'FirstName', 'MiddleName', 'MiddleName', 'LastName', 'O', 'O', 'Mobile', 'Mobile', 'Mobile', 'Mobile', 'Mobile', 'Mobile', 'Mobile', 'Mobile', 'O', 'O', 'O', 'O', 'Email', 'Email', 'Email', 'Email', 'Email', 'Email', 'Email', 'Email', 'Email', 'Email', 'O', 'O', 'O', 'Objective', 'Objective', 'Objective', 'Objective', 'Objective', 'Objective', 'Objective', 'Objective', 'Objective', 'Objective', 'Objective', 'Objective', 'Objective', 'Objective', 'Objective', 'Objective', 'Objective', 'Objective', 'Objective', 'Objective', 'Objective', 'Objective', 'Objective', 'Objective', 'Objective', 'Objective', 'Objective', 'Objective', 'Objective', 'Objective', 'Objective', 'Objective', 'Objective', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 

In [ ]:
b_labels = torch.tensor([item['labels'] for item in batch])
print(b_labels.shape)

torch.Size([8, 1000])


##**Training and Evaluating the model**

In [ ]:
#Clearing GPU ram before training
torch.cuda.empty_cache()

In [ ]:
model.to(DEVICE)
optimizer_grouped_parameters = get_hyperparameters(model, True)
optimizer = Adam(optimizer_grouped_parameters, lr=1.8e-4)

In [ ]:
train_eval_model(
    model,
    TOKENIZER,
    optimizer,
    EPOCHS,
    id2tag,
    tag2id,
    MAX_GRAD_NORM,
    DEVICE,
    train_dl,
    val_dl
)

Epoch:   0%|          | 0/20 [00:00<?, ?it/s]


Train loss: 2.607982873916626
Train accuracy: 0.2658400475183832


Epoch:   5%|▌         | 1/20 [00:12<04:03, 12.81s/it]


Validation loss: 1.9017031843012029
Validation Accuracy: 0.4017547686223274
Classification Report:
                            precision    recall  f1-score   support

          EducationDegree       0.00      0.00      0.00        84
                  Summary       0.45      0.37      0.40      7079
      EducationUniversity       0.00      0.00      0.00        96
                    Email       0.00      0.00      0.00       875
            EducationText       0.53      0.73      0.62      2035
                  JobText       0.00      0.00      0.00       173
            SkillKeywords       0.72      0.03      0.06      3202
           EducationMarks       0.00      0.00      0.00        61
               SkillBlock       0.00      0.00      0.00      2264
ExtraCurricularActivities       0.00      0.00      0.00       255
        EducationLocation       0.00      0.00      0.00        81
                   Mobile       0.00      0.00      0.00        69
               MiddleName  

Epoch:  10%|█         | 2/20 [00:22<03:17, 10.96s/it]

...


Validation loss: 1.5063650499690662
Validation Accuracy: 0.5636968190340427
Classification Report:
                            precision    recall  f1-score   support

                  JobText       0.13      0.09      0.11       231
        EducationLocation       0.47      0.20      0.28       275
            SkillKeywords       0.58      0.10      0.17       819
          EducationDegree       0.28      0.21      0.24       359
                  Summary       0.42      0.82      0.56      7611
      EducationUniversity       0.18      0.83      0.30       405
                    Email       0.93      0.99      0.96       889
            EducationYear       0.64      0.87      0.74       285
       EducationInstitute       0.38      0.05      0.08       549
               SkillBlock       0.77      0.74      0.75      4072
               JobCompany       0.09      0.16      0.11       154
          ProjectDuration       0.00      0.00      0.00        14
           EducationMarks  

Epoch:  95%|█████████▌| 19/20 [03:12<00:09,  9.95s/it]


Validation loss: 1.366641553965482
Validation Accuracy: 0.5859928686776733
Classification Report:
                            precision    recall  f1-score   support

          EducationDegree       0.00      0.00      0.00        84
                  Summary       0.66      0.57      0.61      7079
      EducationUniversity       0.00      0.00      0.00        96
                    Email       0.93      0.98      0.96       875
            EducationText       0.63      0.96      0.76      2035
                  JobText       0.02      0.08      0.04       173
            SkillKeywords       0.52      0.41      0.46      3202
           EducationMarks       0.00      0.00      0.00        61
               SkillBlock       0.45      0.42      0.43      2264
ExtraCurricularActivities       0.35      0.62      0.45       255
        EducationLocation       0.00      0.00      0.00        81
                   Mobile       0.73      0.88      0.80        69
               MiddleName   

Epoch: 100%|██████████| 20/20 [03:22<00:00, 10.11s/it]


Validation loss: 1.542018939148296
Validation Accuracy: 0.5850434494932536
Classification Report:
                            precision    recall  f1-score   support

                  JobText       0.17      0.06      0.09       231
        EducationLocation       0.54      0.27      0.36       275
            SkillKeywords       0.29      0.08      0.12       819
          EducationDegree       0.31      0.19      0.24       359
                  Summary       0.71      0.58      0.64      7611
      EducationUniversity       0.16      0.31      0.21       405
                    Email       0.94      0.98      0.96       889
            EducationYear       0.55      0.90      0.69       285
       EducationInstitute       0.35      0.49      0.41       549
               SkillBlock       0.65      0.74      0.69      4072
               JobCompany       0.12      0.19      0.15       154
          ProjectDuration       0.00      0.00      0.00        14
           EducationMarks   

In [ ]:
#Saving the trained model state
torch.save({"model_state_dict": model.state_dict()},'model-state.bin',)

##**Testing Model**

##**tokenize_resume(data, tokenizer, max_len)**##

this function takes input text data, tokenizes it using a given tokenizer, adds padding and truncation, converts this processed data to pytorch tensors and returns a dictionary containing the tensor objects which can be used by the model.

The function takes **4** parameters as **INPUT**:

* **data**: resume data in the form of a string which is to be tokenized and processed.

* **tokenizer**: This is the given tokenizer which is to be used to tokenize the textual data.

* **max_len**: maximum length of tokenized data.

###**FUNCTION**###

1.  The `tokenizer.encode_plus` function is used to tokenize the given text data and return tokens as well as other information like the attention mask, token type IDs, and offset mapping. this function accepts text (`data`) from the input element alongwith the `max_len` and `return_offsets_mapping=True` and returns a dictionary with elements  -
*    `input_ids` (ids of tokens as returned by tokenizer)
*    `token_type_ids` (which sequence tokens belong to, all 0 in this case)
*    `attention_mask` (describes which tokens the model should consider)

2.  The function then adds padding to each of the output parameters to create equal length arrays.

3. Lastly, the function returns a dictionary (`final_data`) with the elements `input_ids`, `token_type_ids`, `attention_mask` which are pytorch tensors and can be used by the model.

In [ ]:
def tokenize_resume(data, tokenizer, max_len):
  tokenized = tokenizer.encode_plus(
      data,
      max_length= max_len,
      return_offsets_mapping=True)#tokenizing the input text

  padding_length = max_len - len(tokenized['input_ids'])
  res_data = {}
  res_data['input_ids'] = tokenized['input_ids'] + ([0] * padding_length)
  res_data['token_type_ids'] = tokenized['token_type_ids'] + ([0] * padding_length)
  res_data['attention_mask'] = tokenized['attention_mask'] + ([0] * padding_length)

  final_data = {
        'input_ids': torch.tensor(res_data['input_ids'], dtype=torch.long),
        'token_type_ids': torch.tensor(res_data['token_type_ids'], dtype=torch.long),
        'attention_mask': torch.tensor(res_data['attention_mask'], dtype=torch.long),
        'offset_mapping': tokenized['offset_mapping']
    }

  return final_data

##**predict(model, tokenizer, id2tag, device, test_resume, max_len)**##

this function tokenizes the input text, performs inference using the model, extracts entity predictions, and returns a list of predicted entities with their corresponding tags and text.

The function takes **6** parameters as **INPUT**:

* **data**: pre trained model which will be used to make predictions.

* **tokenizer**: This is the given tokenizer which is to be used to tokenize the textual data.

* **id2tag**: a dictionary which maps label ids to their corresponding labels.

* **device**: this specifies the device to be used for computation (cpu or gpu).

* **test_resume**: textual resume data to be processed.

* **max_len**: maximum length of tokenized data.

###**FUNCTION**###

1.  The `tokenizer_resume` function is used to tokenize the given text data and return tokens as well as other information like the attention mask, token type IDs, and offset mapping.

2.  The function then uses the `unsqueeze` function to change the dimension of the input tensors so that they can be used by the model to make predictions. the function also makes a `labels` tensor filled with 1s which is used as a dummy variable for the model.

3. The function then moves the input_ids, input_mask and labels to the device.

4. the torch.no_grad function is used to create a block where no gradients will be computed during forward pass which helps to speed up computation.

5. The function then extracts the outputs from the model in the form of `tmp_eval_loss`(Loss during evaluation) and `logits`(The raw predicted scores for each token and each possible label).

6. The logits tensor is detached from the GPU, moved to the CPU, and converted to a NumPy array. Followed by this the function calculates the predicted label_ids by applying the `argmax` function on the logits array.

7. The function then extracts the predicted label, start index and end index of the text.

8. The function first checks if the predicted labels belng to the restricted labels or not and stores the valid labels and their indexes in the entity list.

9. Finally the function returns the entity list as output.

In [ ]:
def predict(model, tokenizer, id2tag, device, test_resume, max_len):
    model.eval()
    data = tokenize_resume(test_resume, tokenizer, max_len)
    input_ids, input_mask = data['input_ids'].unsqueeze(0), data['attention_mask'].unsqueeze(0)  #changing dimension before passing to model
    labels = torch.tensor([1] * input_ids.size(1),dtype=torch.long).unsqueeze(0)

    input_ids = input_ids.to(device)
    input_mask = input_mask.to(device)
    labels = labels.to(device)

 #passing input to model for classification
    with torch.no_grad():
        outputs = model(
            input_ids,
            token_type_ids=None,
            attention_mask=input_mask,
            labels=labels,
        )
        tmp_eval_loss, logits = outputs[:2]

 #obtaining label ids from model output
    logits = logits.cpu().detach().numpy()
    label_ids = np.argmax(logits, axis=2)

    entities = []
    for label_id, offset in zip(label_ids[0], data['offset_mapping']):
        curr_id = id2tag[label_id] #label name from id
        curr_start = offset[0]
        curr_end = offset[1]
        #checking if entities are restricted labels or not
        if curr_id not in resticted_lables:
            if len(entities) > 0 and entities[-1]['entity'] == curr_id and curr_start - entities[-1]['end'] in [0, 1]:
                entities[-1]['end'] = curr_end
            else:
                entities.append({'entity': curr_id, 'start': curr_start, 'end': curr_end}) #adding entity info to output list

    for ent in entities:
        ent['text'] = test_resume[ent['start']:ent['end']] #extracting entity text using start and end indexes
    return entities

In [ ]:
#Setting prediction model arguments
MAX_LEN = 1000
NUM_LABELS = 66
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
STATE_DICT = torch.load("model-state.bin", map_location=DEVICE)
TOKENIZER = tokenizer
model = AutoModelForTokenClassification.from_pretrained(longformer_model, state_dict=STATE_DICT['model_state_dict'], num_labels=NUM_LABELS)
model.to(DEVICE)

LongformerForTokenClassification(
  (longformer): LongformerModel(
    (embeddings): LongformerEmbeddings(
      (word_embeddings): Embedding(30522, 256, padding_idx=0)
      (token_type_embeddings): Embedding(2, 256)
      (LayerNorm): LayerNorm((256,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
      (position_embeddings): Embedding(1026, 256, padding_idx=0)
    )
    (encoder): LongformerEncoder(
      (layer): ModuleList(
        (0-5): 6 x LongformerLayer(
          (attention): LongformerAttention(
            (self): LongformerSelfAttention(
              (query): Linear(in_features=256, out_features=256, bias=True)
              (key): Linear(in_features=256, out_features=256, bias=True)
              (value): Linear(in_features=256, out_features=256, bias=True)
              (query_global): Linear(in_features=256, out_features=256, bias=True)
              (key_global): Linear(in_features=256, out_features=256, bias=True)
              (

In [ ]:
resume_text = '''PRASHANTH BADALA\nDevops Engineer ,Cloud Engineer -Oracle\n\nHyderabad, Telangana - Email me on Indeed: indeed.com/r/PRASHANTH-BADALA/\nbf4c4b7253a8ece7\n\n• Hands on experience in end-end process of Build Management, Release Management and\nConfiguration Management.\n• Hands on experience on supporting configuration management tools on both physical and cloud\nenvironment\n• Involved in setting up Jenkins in Distributed Environments with Master and Slave\n• Working experience on Subversion (SVN) administration and basic usage\n• Creating Branches, tags and providing SVN user access to all developers in the organization\n• Managing application server instances running on AWS\n• Involved in configuring EC2 instances along with Auto scale up options.\n• Involved in configuration Management using CHEF and automated the complete platform\nsetup.\n• Good knowledge on Cookbooks, Chef Kitchen, Burks file and Metadata.\n• Involved in automating the release using UDeploy and deploying the application to different\nenvironments.\n• Involved in container setup with Docker. Having good understanding of creating and running\nDocker images.\n• Involved in writing Applications, Componenets, Resources and component process flow.\n• Configured Jenkins as contionus integration tool for regular source code builds.\n• Experience in setting up branching strategies, merging and taking regular backups of the source\ncode on SVN server\n• Helping the Developers in SVN related issues\n• Written the integrated build automation scripts using Ant\n• Perform the QA & Stress and UAT builds and deployed the binaries on respective environment\nservers\n• Monitoring the deployment in all the servers\n• Supported setting up of various environments in multi-tier architecture involving load balancers,\nApache webservers, Oracle database,\n• Developed custom scripts to automate the build and release process.\n• Implemented a custom authentication with WebLogic for user authentication, authorization and\npassword policy\n• Monitoring environment using monitoring tools like Wily monitoring tool and custom shell script.\n• Involved in integrating WebLogic with Wily introscope.\n• Involved in resolving tickets with different SLAs like SEV1, SEV2, SEV3 and SEV4.\n• Involved in change management process like opening change records using change\nmanagement tools like INFRA, Managenow.\n• Providing on call, weekend and deployment support.\n• Involved in applying security patches using WebLogic Utility.\n• Deploying applications using different deployment strategies like Stage, No Stage and External\nstage.\nWork Experience:\n\n• Working as Configuration Engineer in Oracle, Hyderabad from July 2015 to till date.\n\nhttps://www.indeed.com/r/PRASHANTH-BADALA/bf4c4b7253a8ece7?isid=rex-download&ikw=download-top&co=IN\nhttps://www.indeed.com/r/PRASHANTH-BADALA/bf4c4b7253a8ece7?isid=rex-download&ikw=download-top&co=IN\n\n\nTechnical Skills:\n\nVersion Control Tools \n\nSubversion(SVN),GIT\n\nWeb/Appservers\nContinuous Integration Tools\nWebLogic 11g, Apache HTTP Server 2.4\nJenkins 1.6, Hudson\n\nBuild Tools \n\nMaven 3\n\nScripting Languages \n\nShell Scripting and Python\n\nOperating Systems \n\nWindows servers […] Windows XP/7, Red Hat LINUX 5\n\nDatabase\n\nOracle 11g\n\nRelease Tools\n\nUDeploy,Jenkins\n\nCloud\n\nAWS\n\nConfiguration Tools\n\nCHEF\n\nEducation Qualification:\nB.Tech From Annamacharya Institute of Technology, JNTU Hyderabad - 2015 \n\nProject Details\n\nTitle: E-Banking solutions \n\nClient : Union Bank of Canada, Canada \nRole : Configuration Engineer \nEnvironment : Maven, Jenkins, CHEF, UDeploy, SVN, Linux, Weblogic,Aws \n\n\n\nDuration : Nov 2016 to till date\n\nProject Description:\n\nThis is a banking project and basic objective of this project is to deal with the loans. This\napplication is based on Java technology. For this we have to schedule the tasks and have to collect\ncode from development team and have to build and deploy the code later have to support the\nrelease management team of which executing Java applications build and deployments in Dev,\nQA, performance and production environments.\n. .\nResponsibilities:\n• Involved in automation of Configuration Management using CHEF and automated multiple\nenvironments like Prod and Non Prod.\n• Involved in configuring AWS Environment to deploy applications.\n• Involved in Release Management and automated the overall release process using Urban Code\nDeployments (UDeploy)\n• For on boarding existing application, performing knowledge transition from development team\nto SCM team on build and deployment process.\n• For new applications, work with development to get the requirements of application build and\ndeployment process.\n• Creating War/Ear files using Ant Script\n• Creating Jenkins/Hudson jobs.\n• Monitor and track requests in Subversion.\n• Monitor and fix the continuous integration builds running in Jenkins.\n• Troubleshooting the compilation errors and provide the technical support to the developers\nteam on that issue.\n• Installation, configuring, administration of Web Logic 9.x, 10.x on Linux environment.\n• Supporting Builds for Multiple environments like Dev, QA, Staging, Perf and Production.\n• Performing Manual and Automation Builds using Maven and Jenkins.\n• Troubleshooting the middle ware issues and resolving the P2,P3 tickets with in SLA\n• Provided on call support on 24/7 basis\n• Involved in creation and configuration of Domains, JVM instances in production, QA and UAT\nenvironments.\n• Configured clusters to provide fail over, load balancing and deployed applications on clusters.\n• Installed, configured and administration to Web logic 10.x, JDBC Data source and Connection\npool configuration with Oracle. Web Logic Administration, Monitoring and Troubleshooting using\nAdmin Console.\n• Have excellent experience in Client Interaction\n• Provided on call support for production tickets on 24/7 basis.\n• Deployment of web and enterprise applications and their updates in dev, production, pre-\nproduction using Admin console.\n\n• Creating branches & merging using Subversion.\n• Performing deployments to multiple environments like Dev, QA, Field, Perf, UAT & Production\nenvs\n• Troubleshooting application related issues by log verification.\n• Writing a UNIX Shell Script and schedule in the respective run levels for automate day-to-day\nactivities such as auto start application server.\n• Automate code deployments by using ANT and Jenkins.\n\n\n\nSprint Value Added Services\n\nClient : Sprint, U.S \nRole : Build and Release Engineer\nEnvironment: Maven, Jenkins, SVN, Linux, Weblogic, Apache,AWS,Docker\nDuration : Dec 2015 to Oct 2016\n\nProject Description:\n\nThis is a Value Added Services provided by Sprint. This application is based on Java technology.\nFor this we have to schedule the tasks and have to collect code from development team and\nhave to build and deploy the code later have to support the release management team of which\nexecuting Java applications build and deployments in Dev, QA, performance and production\nenvironments.\n\nResponsibilities:\n\n• For on boarding existing application, performing knowledge transition from development team\nto SCM team on build and deployment process.\n• For new applications, work with development to get the requirements of application build and\ndeployment process.\n• Installing and configuring Subversion (SVN) and Jenkins.\n• Providing support to Subversion (SVN) related issues.\n• Developing and maintaining build files by using Ant script.\n• Integrate Unit Testing in Ant builds\n• Installation, configuring, administration of Web Logic 9.x, 10.x on Linux environment.\n• Supporting Builds for Multiple environments like Dev, QA, Staging, Perf and Production.\n• Performing Manual and Automation Builds using Maven and Jenkins.\n• Troubleshooting the middle ware issues and resolving the P2,P3 tickets with in SLA\n• Provided on call support on 24/7 basis\n• Involved in creation and configuration of Domains, JVM instances in production, QA and UAT\nenvironments.\n• Configured clusters to provide fail over, load balancing and deployed applications on clusters.\n• Installed, configured and administration to Web logic 10.x, JDBC Data source and Connection\npool configuration with Oracle. Web Logic Administration, Monitoring and Troubleshooting using\nAdmin Console.\n• Have excellent experience in Client Interaction\n• Provided on call support for production tickets on 24/7 basis.\n• Deployment of web and enterprise applications and their updates in dev, production, pre-\nproduction using Admin console.\n• Building the source code using Jenkins.\n• Helped developers in resolving SVN issues and concerns.\n• Responsible for weekly and daily work checks and backups.\n• Environment: WebLogic Application Server 10.3, JDK1.6, Oracle, Apache Webserver, Linux,\nJIRA,Infra,SSH,TOAD\n\nKOhls Retail Services\n\n\n\nClient : Kohls, US\nRole : Build and Release Engineer. \nEnvironment: Maven, Jenkins, SVN, Linux, WebLogic, Apache\nDuration : July 2015 to Nov 2015\n\nProject Description:\n\nThis application is based on Java technology. For this we have to schedule the tasks and have to\ncollect code from development team and have to build and deploy the code later have to support\nthe release management team of which executing Java applications build and deployments in\nDev, QA, performance and production environments.\n\nResponsibilities: \n\n• Developing and maintaining build files by using Ant script.\n• Integrate Unit Testing in Ant builds\n• Installation, configuring, administration of Web Logic 9.x, 10.x on Linux environment.\n• Supporting Builds for Multiple environments like Dev, QA, Staging, Perf and Production.\n\n• Supporting Builds for Multiple environments like Dev, QA, Staging, Perf and Production.\n• Performing Manual and Automation Builds using Maven and Jenkins.\n• Deploying WAR, EAR applications on various targeted servers in the clustered environments.\n• Web Logic Administration, Monitoring and Troubleshooting using Admin Console.\n• Analyzing log files and periodic removal of them\n• Creating branches & merging using Subversion.\n• Performing deployments to multiple environments like Dev, QA, Field, Perf, UAT & Production\nenvs\n• Troubleshooting application related issues by log verification.\n• Writing a UNIX Shell Script and schedule in the respective run levels for automate day-to-day\nactivities such as auto start application server.\n• Automate code deployments by using ANT and Jenkins.\n\n• Involved in changing heap parameters like –Xms, -Xmx, -XnoOpt,-XnoHup etc.\n• Perform daily environment health-check\n• Good in taking the thread dumps and finding the root cause analysis.\n• Created and configured web logic server instances, clusters in domain environment.\n• Installed web logic on production boxes in console mode.\n• Strong experience in administrating by using Admin console.\n\nEnvironment: WebLogic Application Server 9.2, Java, Oracle, Apache Webserver, Linux, JIRA,\nManagenow, Putty, TOAD\n\nWilling to relocate to: Hyderabad, Telangana - Bangalore Urban, Karnataka - Chennai, Tamil Nadu\n\nWORK EXPERIENCE\n\nConfiguration Engineer\n\nOracle -  Hyderabad, Telangana -\n\n\n\nJuly 2015 to Present\n\n• Hands on experience in end-end process of Build Management, Release Management and\nConfiguration Management.\n• Hands on experience on supporting configuration management tools on both physical and cloud\nenvironment\n• Involved in setting up Jenkins in Distributed Environments with Master and Slave\n• Working experience on Subversion(SVN) administration and basic usage\n• Creating Branches, tags and providing SVN user access to all developers in the organization\n• Managing application server instances running on AWS\n• Involved in configuring EC2 instances along with Auto scale up options.\n• Involved in configuration Management using CHEF and automated the complete platform\nsetup.\n• Good knowledge on Cookbooks, Chef Kitchen, Burks file and Metadata.\n• Involved in automating the release using UDeploy and deploying the application to different\nenvironments.\n• Involved in container setup with Docker. Having good understanding of creating and running\nDocker images.\n• Involved in writing Applications, Componenets, Resources and component process flow.\n• Configured Jenkins as contionus integration tool for regular source code builds.\n• Experience in setting up branching strategies, merging and taking regular backups of the source\ncode on SVN server\n• Helping the Developers in SVN related issues\n• Written the integrated build automation scripts using Ant\n• Perform the QA & Stress and UAT builds and deployed the binaries on respective environment\nservers\n• Monitoring the deployment in all the servers\n• Supported setting up of various environments in multi-tier architecture involving load balancers,\nApache webservers, Oracle database, \n• Developed custom scripts to automate the build and release process.\n• Implemented a custom authentication with WebLogic for user authentication, authorization and\npassword policy\n• Monitoring environment using monitoring tools like Wily monitoring tool and custom shell script.\n• Involved in integrating WebLogic with Wily introscope.\n• Involved in resolving tickets with different SLAs like SEV1, SEV2, SEV3 and SEV4.\n• Involved in change management process like opening change records using change\nmanagement tools like INFRA, Managenow.\n• Providing on call, weekend and deployment support.\n• Involved in applying security patches using WebLogic Utility.\n• Deploying applications using different deployment strategies like Stage, No Stage and External\nstage.\n\nConfiguration Engineer\n\nOracle\n\nDevops\n\n\n\nEDUCATION\n\nB.TECH/B.E\n\nAnnamacharya Institute of Technology, JNTU, Hyderabad -  Hyderabad, Telangana\n\n2015\n\nSKILLS\n\nAWS (1 year), CHEF (1 year), Linux (2 years), git, svn, maven, devops, jenkins, Docker,\nweblogic\n\nADDITIONAL INFORMATION\n\nOperating Systems Windows servers […] Windows XP/7, Red Hat LINUX 5\nDatabase Oracle 11g\nRelease Tools UDeploy, Jenkins\nCloud AWS\nConfiguration Tools CHEF
'''

In [ ]:
entities = predict(model, TOKENIZER, id2tag, DEVICE, resume_text, MAX_LEN)
for i in entities:
  print(i["entity"],":",i["text"],"\n")

FullName : PRASHANTH BADALA
Dev 

JobDescription : Hyderabad, Telangana 

Summary : indeed 

WebSite : .com/r/PRASHANTH-BADALA/
bf4c4b7253a8ece 

Summary : 7 

Summary : • Hands on experience in end-end process of Build Management, Release Management and
Configuration Management.
• 

JobDescription : Hands 

Summary : on experience on supporting configuration management tools on both physical and cloud
environment
• Involved in setting up Jenkins in Distributed Environments 

JobDescription : with 

Summary : Master and Slave
• 

JobDescription : Working 

Summary : experience 

JobDescription : on Sub 

Summary : version 

JobDescription : (SVN 

Summary : ) administration and 

JobDescription : basic usage
• Creating 

Summary : Branches 

JobDescription : , tags and providing SVN user access to all developers in the organization
• Managing application server instances running on AWS
• Involved in configuring EC2 instances along with Auto scale up options.
• Involved in configuration